# 🤝 Lab 04: Multi-Agent Systems

**Cieľ:** Pochopiť kedy a ako použiť viac agentov spolupracujúcich na úlohe.

**Čas:** 60 minút

---

## Čo sa naučíš:
- Kedy použiť multi-agent vs single agent
- Orchestrácia vs choreografia
- Manager-Worker pattern

---

In [ ]:
#@title 🔧 Setup
!pip install -q google-generativeai smolagents litellm

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from smolagents import CodeAgent, tool, ManagedAgent
from smolagents.models import LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)

print("✅ Setup OK")

## Časť 1: Kedy Multi-Agent?

### Single Agent je dosť keď:
- Úloha je priamočiara
- Všetky nástroje súvisia s jednou doménou
- Nepotrebuješ špecializáciu

### Multi-Agent keď:
- Úloha má viac nezávislých častí
- Rôzne časti vyžadujú rôznu expertízu
- Chceš paralelizovať prácu
- Potrebuješ kontrolu a review

## Časť 2: Manager-Worker Pattern

```
┌──────────────────┐
│  MANAGER AGENT   │ ← Rozhoduje, deleguje, kombinuje
└────────┬─────────┘
         │
    ┌────┴────┬────────────┐
    │         │            │
┌───┴───┐ ┌───┴───┐ ┌──────┴──────┐
│Worker │ │Worker │ │   Worker    │
│  A    │ │  B    │ │     C       │
└───────┘ └───────┘ └─────────────┘
Research   Writer    Fact-checker
```

In [ ]:
#@title 👷 Worker Agents: Špecializovaní agenti

# Worker 1: Research Agent
@tool
def search_web(query: str) -> str:
    """Vyhľadá informácie na webe (simulované)."""
    # Demo dáta
    data = {
        "ai trends 2026": "Top AI trendy 2026: 1) Agentic AI, 2) Multimodal models, 3) Edge AI, 4) AI governance",
        "python popularity": "Python zostáva #1 jazyk pre AI/ML s 30% rastom popularity.",
        "llm costs": "Náklady na LLM inference klesli o 90% za posledné 2 roky."
    }
    for key, value in data.items():
        if key in query.lower():
            return value
    return "Žiadne výsledky pre: " + query

research_agent = CodeAgent(
    tools=[search_web],
    model=model,
    max_steps=3,
    name="research_agent",
    description="Špecialista na vyhľadávanie informácií a research."
)

print("✅ Research Agent vytvorený")

In [ ]:
#@title 👷 Worker 2: Writer Agent

@tool
def format_as_bullet_points(text: str) -> str:
    """Preformátuje text na bullet points."""
    lines = text.split('.')
    bullets = "\n".join([f"• {line.strip()}" for line in lines if line.strip()])
    return bullets

@tool
def create_summary(text: str, max_words: int = 50) -> str:
    """Vytvorí stručné zhrnutie textu."""
    words = text.split()
    if len(words) <= max_words:
        return text
    return ' '.join(words[:max_words]) + '...'

writer_agent = CodeAgent(
    tools=[format_as_bullet_points, create_summary],
    model=model,
    max_steps=3,
    name="writer_agent",
    description="Špecialista na písanie a formátovanie textu."
)

print("✅ Writer Agent vytvorený")

In [ ]:
#@title 👔 Manager Agent

# Zabalíme workery ako ManagedAgents
managed_research = ManagedAgent(
    agent=research_agent,
    name="researcher",
    description="Vyhľadá a získa informácie na danú tému."
)

managed_writer = ManagedAgent(
    agent=writer_agent,
    name="writer",
    description="Napíše a naformátuje text na základe informácií."
)

# Manager agent - riadi ostatných
manager_agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=10,
    managed_agents=[managed_research, managed_writer],
    system_prompt="""
    Si manažér tímu. Máš k dispozícii:
    - researcher: vyhľadá informácie
    - writer: napíše a naformátuje text
    
    Pre každú úlohu:
    1. Najprv použi researcher na získanie informácií
    2. Potom použi writer na vytvorenie finálneho outputu
    3. Skombinuj výsledky do ucelenej odpovede
    """
)

print("✅ Manager Agent vytvorený")
print("   Riadi: researcher, writer")

In [ ]:
#@title ▶️ Test Multi-Agent systému

task = "Priprav krátky prehľad AI trendov pre rok 2026 vo forme bullet points."

print(f"📋 Úloha: {task}")
print("=" * 50)

result = manager_agent.run(task)

print("=" * 50)
print(f"\n✅ Výsledok:\n{result}")

## Časť 3: Praktický príklad - Content Pipeline

In [ ]:
#@title 📰 Content Creation Pipeline

# Simulovaná databáza článkov
ARTICLES_DB = []

# --- RESEARCH AGENT ---
@tool
def gather_topic_info(topic: str) -> str:
    """Zhromaždí informácie o téme."""
    info = {
        "ai": "AI transformuje priemysel. Kľúčové oblasti: automatizácia, predikcia, personalizácia.",
        "cloud": "Cloud computing rastie 20% ročne. Hlavní hráči: AWS, Azure, GCP.",
        "security": "Kybernetické útoky narástli o 40%. Zero trust je nový štandard."
    }
    for key, value in info.items():
        if key in topic.lower():
            return f"Research pre '{topic}':\n{value}"
    return f"Téma '{topic}' vyžaduje ďalší research."

researcher = CodeAgent(
    tools=[gather_topic_info],
    model=model,
    max_steps=3,
    name="researcher",
    description="Robí research na témy"
)

# --- WRITER AGENT ---
@tool
def write_article(title: str, content: str) -> str:
    """Napíše článok s daným titulkom a obsahom."""
    article = f"""
# {title}

{content}

---
*Generované AI Academy*
    """
    return article.strip()

writer = CodeAgent(
    tools=[write_article],
    model=model,
    max_steps=3,
    name="writer",
    description="Píše články"
)

# --- EDITOR AGENT ---
@tool
def check_quality(text: str) -> str:
    """Skontroluje kvalitu textu."""
    issues = []
    
    if len(text) < 100:
        issues.append("Text je príliš krátky")
    if text.count('#') == 0:
        issues.append("Chýba nadpis")
    if '.' not in text:
        issues.append("Chýba interpunkcia")
    
    if issues:
        return "❌ Problémy: " + ", ".join(issues)
    return "✅ Kvalita OK"

@tool
def publish_article(article: str) -> str:
    """Publikuje článok do databázy."""
    ARTICLES_DB.append(article)
    return f"✅ Článok publikovaný! (celkom {len(ARTICLES_DB)} článkov)"

editor = CodeAgent(
    tools=[check_quality, publish_article],
    model=model,
    max_steps=3,
    name="editor",
    description="Kontroluje a publikuje články"
)

print("✅ Content Pipeline agenti vytvorení")

In [ ]:
#@title 👔 Content Manager

content_manager = CodeAgent(
    tools=[],
    model=model,
    max_steps=15,
    managed_agents=[
        ManagedAgent(researcher, "researcher", "Získa info o téme"),
        ManagedAgent(writer, "writer", "Napíše článok"),
        ManagedAgent(editor, "editor", "Skontroluje a publikuje")
    ],
    system_prompt="""
    Si content manager. Proces vytvorenia článku:
    
    1. RESEARCH: Použi 'researcher' na získanie info o téme
    2. WRITE: Použi 'writer' na napísanie článku
    3. REVIEW: Použi 'editor' na kontrolu kvality
    4. PUBLISH: Ak je kvalita OK, použi 'editor' na publikovanie
    
    Ak editor nájde problémy, vráť sa k writerovi.
    """
)

print("✅ Content Manager vytvorený")

In [ ]:
#@title ▶️ Vytvorenie článku

result = content_manager.run("""
Vytvor krátky článok o AI v enterprise prostredí.
Článok by mal byť informatívny a profesionálny.
""")

print("=" * 50)
print("VÝSLEDOK:")
print("=" * 50)
print(result)

## Časť 4: Patterns a Anti-Patterns

### ✅ Dobré patterns:

1. **Manager-Worker** - jeden agent koordinuje špecializovaných
2. **Pipeline** - sekvenčné spracovanie (research → write → review)
3. **Peer Review** - agenti si navzájom kontrolujú prácu

### ❌ Anti-patterns:

1. **Too many agents** - zbytočná komplexita
2. **Circular dependencies** - agenti sa volajú navzájom donekonečna
3. **No clear ownership** - nikto nie je zodpovedný za výsledok

In [ ]:
#@title ❌ Anti-pattern: Príliš veľa agentov

# Toto je ZLÝM príkladom - nepotrebujeme 5 agentov na jednoduchú úlohu!

# Agent 1: Čítač otázky
# Agent 2: Analyzátor kontextu  
# Agent 3: Vyhľadávač
# Agent 4: Formulovač odpovede
# Agent 5: Kontrolór gramatiky

# Lepšie: Jeden agent s dobrými nástrojmi!

print("❌ Nepoužívaj 5 agentov keď stačí 1 s dobrými nástrojmi!")

---

## 🏋️ Cvičenie: Customer Service Multi-Agent

**Úloha:** Vytvor multi-agent systém pre customer service:

1. **Triage Agent** - kategorizuje požiadavky (billing, technical, general)
2. **Billing Agent** - rieši fakturačné otázky
3. **Technical Agent** - rieši technické problémy
4. **Manager Agent** - koordinuje a eskaluje

In [ ]:
#@title ✏️ Tvoje riešenie

# TODO: Implementuj agentov

# Triage tools
@tool
def categorize_request(request: str) -> str:
    """Kategorizuje zákaznícku požiadavku."""
    # TODO: Implementuj
    pass

# Billing tools
@tool
def check_invoice(invoice_id: str) -> str:
    """Skontroluje faktúru."""
    # TODO: Implementuj
    pass

# Technical tools
@tool
def diagnose_issue(description: str) -> str:
    """Diagnostikuje technický problém."""
    # TODO: Implementuj
    pass

# TODO: Vytvor agentov a manager agent

# Test:
# print(manager.run("Moja faktúra č. 12345 je nesprávna"))
# print(manager.run("Aplikácia mi padá pri prihlásení"))

---

## 📝 Reflexia

1. **Kedy je multi-agent lepší než single agent?**

2. **Aké sú nevýhody multi-agent systémov?** (Hint: latencia, cena, komplexita)

3. **Ako by si navrhol multi-agent pre svoj reálny projekt?**

---

## 🎓 Gratulujem!

Dokončil si základné laby AI Academy!

### Čo si sa naučil:
- ✅ Základy AI agentov (Lab 01)
- ✅ Nástroje pre agentov (Lab 02)  
- ✅ RAG pattern (Lab 03)
- ✅ Multi-agent systémy (Lab 04)

### Ďalšie kroky:
- Deployment na HuggingFace Spaces
- Team project
- Hackathon!